<a href="https://colab.research.google.com/github/alexlinapp/python_tools_practice/blob/main/resNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchinfo

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from fastai.vision.all import *
import torch.nn.functional as F
from torchinfo import summary
print(torch.__version__)

2.6.0+cu124


In [3]:
def get_dls(bs, size, shuffle=True):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs, shuffle=shuffle)
def get_data(url, presize, resize, bs=64):
    path = untar_data(url)
    return DataBlock(
        blocks=(ImageBlock, CategoryBlock), get_items=get_image_files,
        splitter=GrandparentSplitter(valid_name='val'),
        get_y=parent_label, item_tfms=Resize(presize),
        batch_tfms=[*aug_transforms(min_scale=0.5, size=resize),
                    Normalize.from_stats(*imagenet_stats)],
    ).dataloaders(path, bs=bs)

In [5]:
dls = get_data(URLs.IMAGENETTE_320, 160, 224, 128)
xb, yb = dls.one_batch()
xb.shape, yb.shape, type(dls), type(dls.train)

(torch.Size([128, 3, 224, 224]),
 torch.Size([128]),
 fastai.data.core.DataLoaders,
 fastai.data.core.TfmdDL)

In [6]:
def conv_layer(in_channels, out_channels, act_cls = nn.ReLU(), ks=3, stride=2, padding=0):
  layers = [nn.Conv2d(in_channels, out_channels, kernel_size=ks, stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels)]
  if act_cls is not None: layers.append(act_cls)
  return nn.Sequential(*layers)

def bottleneck_block1(in_channels, out_channels, ks=3, act_cls = nn.ReLU(), stride=1):
  return nn.Sequential(conv_layer(in_channels, out_channels=out_channels//4, act_cls=act_cls, ks=1, stride=1),
                       conv_layer(in_channels=out_channels//4, out_channels=out_channels//4, act_cls=act_cls, ks=ks, stride=stride, padding=ks//2),
                       conv_layer(in_channels=out_channels//4, out_channels=out_channels, act_cls=None, ks=1, stride=1))

class ResBlock(nn.Module):
  def __init__(self, in_channels, out_channels, ks=3, stride=2):
    super().__init__()
    self.conv_block1 = nn.Sequential(conv_layer(in_channels, out_channels, ks=ks, stride=stride, padding=ks//2),
                                     conv_layer(out_channels, out_channels, act_cls = None, ks=ks, stride=1, padding=ks//2))
    self.id_conv = nn.Identity() if in_channels==out_channels and stride == 1 else conv_layer(in_channels, out_channels, act_cls = None, ks=1, stride=stride)
  def forward(self, x):
    result = self.conv_block1(x)
    identity = self.id_conv(x)
    return F.relu(result + identity)

def ResBlock2(in_channels, out_channels, ks=3, stride=2):
  return nn.Sequential(ResBlock(in_channels, out_channels, ks=ks, stride=stride),
                       ResBlock(out_channels, out_channels, ks=ks, stride=1))

class ResBlock3(nn.Module):
  def __init__(self, in_channels, out_channels, ks=3, stride=2):
    super().__init__()
    self.conv_block1 = bottleneck_block1(in_channels, out_channels, ks=ks, act_cls = nn.ReLU(), stride=stride)
    self.id_conv = nn.Identity() if in_channels==out_channels and stride==1 else conv_layer(in_channels, out_channels, act_cls = None, ks=1, stride=stride)
  def forward(self, x):
    return F.relu(self.conv_block1(x) + self.id_conv(x))


def resnet_stem(*sizes):
  # use max pooling to retain general/maximal features
  return [conv_layer(sizes[i], sizes[i+1], ks=3, stride=2, padding=1) for i in range(len(sizes)-1)] + [nn.MaxPool2d(kernel_size=3, stride=2, padding=1)]

class ResNet5(nn.Sequential):
  def __init__(self, stage_layers, in_channels=3, num_classes=10, expansion=1):
    stem = resnet_stem(in_channels, 32, 32, 64)
    self.channels = [64, 64, 128, 256, 512]
    for i in range(1, 5): self.channels[i] *= expansion
    stages = [self.make_layer(*o) for o in enumerate(stage_layers)]
    super().__init__(*stem, *stages, nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.Linear(self.channels[-1], num_classes))


  # custom function. Not using forward here since directly inheriting from nn.Sequential and not nn.Module
  # Multiple ResBlocks per stage, input layer of stage uses ch_in, but then intermediate layers use ch_out and output layer is ch_out
  def make_layer(self, idx, n_layers):
    stride = 1 if idx == 0 else 2
    ch_in, ch_out = self.channels[idx:idx+2]
    return nn.Sequential(*[ResBlock(ch_in if i==0 else ch_out, ch_out, stride=stride if i==0 else 1) for i in range(n_layers)])






class ResNet(nn.Module):
  def __init__(self, in_channels=3, num_classes=10):
    super().__init__()
    self.block1 = nn.Sequential(conv_layer(in_channels, 16),
                                conv_layer(16, 32),
                                conv_layer(32, 64),
                                conv_layer(64, 128),
                                conv_layer(128, 256))

    # self.layer1 = nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=(3, 3), stride=2)
    # self.relu = nn.ReLU()
    # self.layer2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=2)
    # self.layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=2)
    # self.layer4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=2)
    # self.layer5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=2)
    #self.layer6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), stride=2)
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.lastlayer = nn.Linear(256, num_classes)

  def forward(self, x):
    # x = self.layer1(x)
    # x = self.relu(x)
    # x = self.layer2(x)
    # x = self.relu(x)
    # x = self.layer3(x)
    # x = self.relu(x)
    # x = self.layer4(x)
    # x = self.relu(x)
    # x = self.layer5(x)
    # x = self.relu(x)
    x = self.block1(x)
    x = self.pool(x)
    x = x.view(x.size(0), -1)
    return self.lastlayer(x)

class ResNet2(nn.Module):
  def __init__(self, in_channels=3, num_classes=10):
    super().__init__()
    self.block1 = ResBlock(in_channels, 16)
    self.block2 = ResBlock(16, 32)
    self.block3 = ResBlock(32, 64)
    self.block4 = ResBlock(64, 128, stride=1)
    self.block5 = ResBlock(128, 256, stride=1)
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.lastlayer = nn.Linear(256, num_classes)
  def forward(self, x):
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.block5(x)
    x = self.pool(x)
    x = x.view(x.size(0), -1)
    return self.lastlayer(x)

class ResNet3(nn.Module):
  def __init__(self, in_channels=3, num_classes=10):
    super().__init__()
    self.block1 = ResBlock2(in_channels, 16)
    self.block2 = ResBlock2(16, 32)
    self.block3 = ResBlock2(32, 64)
    self.block4 = ResBlock2(64, 128)
    self.block5 = ResBlock2(128, 256)
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.lastlayer = nn.Linear(256, num_classes)
  def forward(self, x):
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.block5(x)
    x = self.pool(x)
    x = x.view(x.size(0), -1)
    return self.lastlayer(x)

class ResNet4(nn.Module):
  def __init__(self, in_channels=3, num_classes=10):
    super().__init__()
    self.block1 = ResBlock2(in_channels, 16, stride=2)
    self.block2 = ResBlock2(16, 32, stride=1)
    self.block3 = ResBlock2(32, 64, stride=2)
    self.block4 = ResBlock2(64, 128, stride=2)
    self.block5 = ResBlock3(128, 256, stride=2)
    self.block6 = ResBlock3(256, 512, stride=2)
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.lastlayer = nn.Linear(512, num_classes)
  def forward(self, x):
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    #print(x.shape)
    x = self.block4(x)
    #print(x.shape)
    x = self.block5(x)
    #print(x.shape)
    x = self.block6(x)
    #print(x.shape)
    x = self.pool(x)
    x = x.view(x.size(0), -1)
    return self.lastlayer(x)

print(torch.cuda.is_available())

True


In [ ]:
model1 = ResNet(3, 10)
#ummary(model1, input_size=(64, 3, 128, 128))
model2 = ResNet2(3, 10)
#summary(model2, input_size=(64, 3, 128, 128))
model3 = ResNet3(3, 10)
summary(model3, input_size=(64, 3, 128, 128))
model4 = ResNet4(3, 10)
summary(model4, input_size=(64, 3, 128, 128))
model5 = ResNet5(3, 10)
summary(model5, input_size=(64, 3, 224, 224))

In [10]:
def validate_model(model, loss_fn, valid_loader, epochs = 1, device="cpu"):
  model.to(device)
  model.eval()
  for epoch in range(epochs):
    correct = 0
    val_loss = 0
    with torch.no_grad():
      for xb, yb in valid_loader:
        xb = xb.to(device)
        yb = Tensor(yb).to(device)
        preds = model(xb)
        loss = loss_fn(preds, yb)
        val_loss += loss.item() * xb.size(0)
        correct += (preds.argmax(dim=1) == yb).sum().item()
    val_acc = correct / len(valid_loader.dataset)
    val_loss /= len(valid_loader.dataset)
    print(f"Epoch {epoch+1}: Val loss {val_loss:.4f}, Val accuracy {val_acc:.4f}")
def train_model(model, optimizer, loss_fn, train_loader, epochs=3, device="cpu"):
  model.to(device)
  model.train()
  for epoch in range(epochs):
    train_loss = 0
    train_accruracy = 0
    train_correct = 0
    i = 0
    for xb, yb in train_loader:
      xb = Tensor(xb).to(device)
      yb = Tensor(yb).to(device)
      optimizer.zero_grad()
      pred = model(xb)          # (batch_size, num_classes)
      loss = loss_fn(pred, yb)
      loss.backward()
      optimizer.step()
      train_loss += loss.item() * xb.size(0)
      train_accruracy += (pred.argmax(dim=1) == yb).float().mean().item()
      train_correct += (pred.argmax(dim=1) == yb).sum().item()
      i += 1
      # if (i >= 50): break

    avg_loss = train_loss / (i * train_loader.bs)
    avg_acc = train_correct / (i * train_loader.bs)
    print(f"Epoch: {epoch+1}/{epochs}, Loss: {avg_loss}, Accuracy: {avg_acc}")
    print(i)
    # validate_model(model, loss_fn, dls.valid, epochs = 1, device="cuda")
  model.eval()
  correct = 0
  val_loss = 0
  with torch.no_grad():
    for xb, yb in valid_loader:
      xb = xb.to(device)
      yb = Tensor(yb).to(device)
      preds = model(xb)
      loss = loss_fn(preds, yb)
      val_loss += loss.item() * xb.size(0)
      correct += (preds.argmax(dim=1) == yb).sum().item()
  val_acc = correct / len(valid_loader.dataset)
  val_loss /= len(valid_loader.dataset)
  print(f"Epoch {epoch+1}/{epochs}: Val loss {val_loss:.4f}, Val accuracy {val_acc:.4f}")

def train_validate_model(model, optimizer, loss_fn, dls, epochs=3, device="cpu"):
  model.to(device)
  for epoch in range(epochs):
    train_loss = 0
    train_accruracy = 0
    train_correct = 0
    i = 0
    model.train()
    for xb, yb in dls.train:
      xb = Tensor(xb).to(device)
      yb = Tensor(yb).to(device)
      optimizer.zero_grad()
      pred = model(xb)          # (batch_size, num_classes)
      loss = loss_fn(pred, yb)
      loss.backward()
      optimizer.step()
      train_loss += loss.item() * xb.size(0)
      train_accruracy += (pred.argmax(dim=1) == yb).float().mean().item()
      train_correct += (pred.argmax(dim=1) == yb).sum().item()
      i += 1
      # if (i >= 50): break

    avg_loss = train_loss / len(dls.train.dataset)
    avg_acc = train_correct / len(dls.train.dataset)
    print(f"Epoch: {epoch+1}/{epochs}, Loss: {avg_loss}, Accuracy: {avg_acc}")
    print(i)
    # validate_model(model, loss_fn, dls.valid, epochs = 1, device="cuda")
    model.eval()
    correct = 0
    val_loss = 0
    with torch.no_grad():
      for xb, yb in dls.valid:
        xb = xb.to(device)
        yb = Tensor(yb).to(device)
        preds = model(xb)
        loss = loss_fn(preds, yb)
        val_loss += loss.item() * xb.size(0)
        correct += (preds.argmax(dim=1) == yb).sum().item()
    val_acc = correct / len(dls.valid.dataset)
    val_loss /= len(dls.valid.dataset)
    print(f"Epoch {epoch+1}/{epochs}: Val loss {val_loss:.4f}, Val accuracy {val_acc:.4f}")

In [11]:
model1 = ResNet(3, 10) #original lr = 3e-4
train_validate_model(model1, torch.optim.Adam(model1.parameters(), lr=0.001), nn.CrossEntropyLoss(), dls, 5, "cuda")

Epoch: 1/5, Loss: 1.7470849347676052, Accuracy: 0.3978244798817193
73
Epoch 1/5: Val loss 1.7537, Val accuracy 0.4107
Epoch: 2/5, Loss: 1.4274018610250183, Accuracy: 0.5241313760692787
73
Epoch 2/5: Val loss 1.3971, Val accuracy 0.5437
Epoch: 3/5, Loss: 1.261843135155724, Accuracy: 0.5740838525715493
73
Epoch 3/5: Val loss 1.6067, Val accuracy 0.5009
Epoch: 4/5, Loss: 1.1620869831419982, Accuracy: 0.6116802196641673
73
Epoch 4/5: Val loss 1.3453, Val accuracy 0.5603
Epoch: 5/5, Loss: 1.0936422224142495, Accuracy: 0.6367092618016686
73
Epoch 5/5: Val loss 1.2109, Val accuracy 0.6150


In [12]:
model5v1 = ResNet5([1,1,1,1])
train_validate_model(model5v1, torch.optim.Adam(model5v1.parameters(), lr=0.001), nn.CrossEntropyLoss(), dls, 15, "cuda")

Epoch: 1/15, Loss: 1.633667902654901, Accuracy: 0.42507128524659415
73
Epoch 1/15: Val loss 1.2856, Val accuracy 0.5776
Epoch: 2/15, Loss: 1.2342402983623266, Accuracy: 0.5734502059351568
73
Epoch 2/15: Val loss 1.6106, Val accuracy 0.4981
Epoch: 3/15, Loss: 1.090274965892763, Accuracy: 0.6312176576196008
73
Epoch 3/15: Val loss 1.3255, Val accuracy 0.5839
Epoch: 4/15, Loss: 0.9805615959203738, Accuracy: 0.6635336360756152
73
Epoch 4/15: Val loss 1.3098, Val accuracy 0.5992
Epoch: 5/15, Loss: 0.8984631759864323, Accuracy: 0.692153342486007
73
Epoch 5/15: Val loss 1.1372, Val accuracy 0.6502
Epoch: 6/15, Loss: 0.8728993704180769, Accuracy: 0.7002851409863766
73
Epoch 6/15: Val loss 1.0260, Val accuracy 0.6688
Epoch: 7/15, Loss: 0.7905895818512321, Accuracy: 0.728271200760376
73
Epoch 7/15: Val loss 1.0208, Val accuracy 0.6734
Epoch: 8/15, Loss: 0.7554168507798268, Accuracy: 0.739888055760904
73
Epoch 8/15: Val loss 1.0185, Val accuracy 0.6871
Epoch: 9/15, Loss: 0.7380894443303684, Accur

In [14]:
model5v2 = ResNet5([2,2,2,2])
train_validate_model(model5v2, torch.optim.Adam(model5v2.parameters(), lr=0.001), nn.CrossEntropyLoss(), dls, 15, "cuda")

Epoch: 1/15, Loss: 1.719163178163819, Accuracy: 0.39138240574506283
73
Epoch 1/15: Val loss 1.5611, Val accuracy 0.4889
Epoch: 2/15, Loss: 1.316360742744416, Accuracy: 0.5545464146161158
73
Epoch 2/15: Val loss 1.4427, Val accuracy 0.5496
Epoch: 3/15, Loss: 1.1471007122813692, Accuracy: 0.6100961030731862
73
Epoch 3/15: Val loss 1.4288, Val accuracy 0.5659
Epoch: 4/15, Loss: 1.0452342313878678, Accuracy: 0.6438905903474496
73
Epoch 4/15: Val loss 1.5034, Val accuracy 0.5518
Epoch: 5/15, Loss: 0.9357773529739211, Accuracy: 0.6822262118491921
73
Epoch 5/15: Val loss 1.4105, Val accuracy 0.5605
Epoch: 6/15, Loss: 0.9092787866393998, Accuracy: 0.6855000528038864
73
Epoch 6/15: Val loss 0.9925, Val accuracy 0.6734
Epoch: 7/15, Loss: 0.8640809787819134, Accuracy: 0.702397296441018
73
Epoch 7/15: Val loss 0.9143, Val accuracy 0.7065
Epoch: 8/15, Loss: 0.8209133641698919, Accuracy: 0.7174992079417045
73
Epoch 8/15: Val loss 1.1074, Val accuracy 0.6512
Epoch: 9/15, Loss: 0.7754360694091617, Acc

KeyboardInterrupt: 